In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint
import json
import glob, os

# Google developer API key
from api_config import yelpkey

ModuleNotFoundError: No module named 'api_config'

In [3]:
# Output File (CSV)

stl_zip_codes = "STL_zipcodesCORRECT.csv"

stl_zip_df = pd.read_csv(stl_zip_codes)
stl_zip_df.head()

,ZIP,Code,Type,County,Population,Area,Code(s),Unnamed: 7,Unnamed: 8
0,ZIP,Code,63101,Standard,Saint,Louis,City,2620,314
1,ZIP,Code,63102,Standard,Saint,Louis,City,2316,314
2,ZIP,Code,63103,Standard,Saint,Louis,City,6900,314
3,ZIP,Code,63104,Standard,Saint,Louis,City,18656,314
4,ZIP,Code,63106,Standard,Saint,Louis,City,11883,314


In [4]:
stl_zip_df.drop(['ZIP', 'Code', 'County', 'Population', 'Area', 'Code(s)', 'Unnamed: 8'], axis=1, inplace=True)
stl_zip_df.head()

,Type,Unnamed: 7
0,63101,2620
1,63102,2316
2,63103,6900
3,63104,18656
4,63106,11883


In [5]:
stl_zip_df = stl_zip_df.rename(columns={"Type": "ZIP Code", "Unnamed: 7": "Population"})
stl_zip_df.head()

,ZIP Code,Population
0,63101,2620
1,63102,2316
2,63103,6900
3,63104,18656
4,63106,11883


In [6]:
stl_zip_df.dtypes

ZIP Code      int64
Population    int64
dtype: object

In [7]:
zip_codes = stl_zip_df['ZIP Code']
#zip_codes

In [8]:
# importing crime data (CSV) and combining into 1 df

stl_crime_df = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "crime_data/complete_file.csv"))))
stl_crime_df.head()

,Complaint,CodedMonth,DateOccur,FlagCrime,FlagUnfounded,FlagAdministrative,Count,FlagCleanup,Crime,District,...,LocationName,LocationComment,CADAddress,CADStreet,Unnamed: 18,state,street address,Zip,XCoord,YCoord
0,19-025935,2019-06,1/1/1900 0:00,Y,,,1,,51322,4,...,,,1811,WARREN,St. Louis,MO,"1811 WARREN, St. Louis, MO",63106,898905.7,1036540.0
1,19-020007,2019-05,6/18/1994 0:01,Y,,,1,,175500,1,...,,,7300,VERMONT,St. Louis,MO,"7300 VERMONT, St. Louis, MO",63101,892793.9,1076992.0
2,19-010094,2019-03,2/7/1996 13:02,Y,,,1,,263899,4,...,,@HEADQUARTERS,1915,OLIVE,St. Louis,MO,"1915 OLIVE, St. Louis, MO",63103,0.0,0.0
3,19-007109,2019-02,10/7/1996 12:43,Y,,,1,,263899,4,...,,@HEADQUARTERS,1915,OLIVE,St. Louis,MO,"1915 OLIVE, St. Louis, MO",63103,0.0,0.0
4,19-028394,2019-06,7/5/1998 12:00,Y,,,1,,115400,6,...,,,5704,ACME,St. Louis,MO,"5704 ACME, St. Louis, MO",63101,0.0,0.0


In [9]:
stl_crime_df.drop(['FlagCrime', 'FlagUnfounded', 'FlagAdministrative', 'Count', 'FlagCleanup'], axis=1, inplace=True)
#stl_crime_df.head()

In [10]:
zip_code_df = pd.DataFrame(stl_crime_df["Zip"].value_counts())
zip_code_df.head()

,Zip
63101,15623
63116,1156
63118,989
63147,859
63103,779


In [11]:
new_zip = zip_code_df.reset_index()
clean_zip = new_zip.rename(columns={"index": "ZIP Code", "Zip": "Crimes Committed"})
clean_zip.head()

,ZIP Code,Crimes Committed
0,63101,15623
1,63116,1156
2,63118,989
3,63147,859
4,63103,779


In [13]:
crime_merge = pd.merge(clean_zip, stl_zip_df, on="ZIP Code", how="outer")
crime_merge.head()

,ZIP Code,Crimes Committed,Population
0,63101,15623.0,2620.0
1,63116,1156.0,43540.0
2,63118,989.0,26704.0
3,63147,859.0,11373.0
4,63103,779.0,6900.0


In [14]:
clean_zip_df = crime_merge.dropna(subset=['Crimes Committed', 'Population'])
clean_zip_df

,ZIP Code,Crimes Committed,Population
0,63101,15623.0,2620.0
1,63116,1156.0,43540.0
2,63118,989.0,26704.0
3,63147,859.0,11373.0
4,63103,779.0,6900.0
5,63111,748.0,20313.0
6,63104,666.0,18656.0
7,63107,650.0,11912.0
8,63115,638.0,20775.0
9,63112,574.0,20368.0


In [15]:
clean_zip_df.sum()

ZIP Code            1136041.0
Crimes Committed      26317.0
Population           309233.0
dtype: float64

In [ ]:
# running thru yelp for liquor stores by zip (see homework and activity 1-9)
#count = 1
#for zip in zipcodes:
#    print(f"Making request number: {count} - {zip}")
#    zip_response_json = reqests.get(query_url + zip_code).json()
#    count=count+1
#    try:
#        result_liqour_count.append(zip_response_json['<main group>'][<sub group>])
#        pprint(zip_response_json)
#    except KeyError:
#        print("Problem with Zip Code.  Skipping...")
#        continue
    


In [ ]:
#crime_zip = "crime_data/complete_file.csv"

#crime_zip_df = pd.read_csv(crime_zip)
#crime_zip_df.head()

In [16]:



# **************************

# Brooklyn code:
#r=requests.get("https://api.yelp.com/v3/businesses/search", headers={"content-type":"Bearer"})

url = "https://api.yelp.com/v3/businesses/search"

addresses_df = pd.read_csv("addresszip.csv")


store_names = []
loc_zip_codes =[]
stl_zip_codes =[]




unique_zip_codes = addresses_df["Zip"].unique()


for zip_code in ["63103","63104"]:
    params = {
        "Authorization": yelpkey,
        "location": zip_code,
        "term": "liquor_store",
        "sort_by": "distance",
        "limit": "50"
    }

    response = requests.get(url, params=params, headers={"Authorization": f"Bearer {yelpkey}"})
    
    data = response.json()
    print('input zip {zip}'.format(zip=zip_code))
    
    for store in data["businesses"]:
        loc_zip = store["location"]["zip_code"]
        store_name = store["name"]
        if loc_zip == zip_code:
            loc_zip_codes.append(loc_zip)
            store_names.append(store_name)
            stl_zip_codes.append(zip_code)
        
        
#         combined_stores.append(pd.Series([loc_data,name_data],index=["Zip Code","Store Name"]),ignore_index=True)
        
        
    
combined_stores = pd.DataFrame({"STL Zip Code":stl_zip_codes ,"Store Name": store_names, "Store Zip Code": loc_zip_codes})
combined_stores

#zip_code = "63115"

# response = requests.get(url, params=params, headers={"Authorization": f"Bearer {yelpkey}"})
# response
#unique_zip_codes

NameError: name 'yelpkey' is not defined

In [ ]:
data = response.json()
pprint(data)

In [ ]:
for store in data["businesses"]:
    
    print(store["location"]["zip_code"])